In [ ]:
from glob import glob
from PIL import Image
import numpy as np

path = []
path = path + ['data/DeepQ-Synth-Hand-0%d/data/s00%d' % (i, j) for i, j in
           [(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9)]]
path = path + ['data/DeepQ-Vivepaper/data/air', 'data/DeepQ-Vivepaper/data/book']
path_img = [ p + '/img/*.png' for p in path ]
path_img = sum([glob(p) for p in path_img], [])

path_label = [ p + '/label/*.json' for p in path ]
path_label = sum([glob(p) for p in path_label], [])

path_label.sort()
path_img.sort()
pathes = list(zip(path_img, path_label))

In [ ]:
def xy_flip(j, im):
    if len(j)==4:
        return [im.size[0]-j[2], j[1], im.size[0]-j[0], j[3]]
    else:
        return j
def label_flip(k):
    if k in ['L', 'R']:
        return 'L' if k=='R' else 'R'
    return k
def json_flip(j, im):
    return {label_flip(k) : (json_flip if type(v) is dict else xy_flip)(v, im) for k, v in j.items()}

In [ ]:
import json
import pathlib
from PIL import Image
from os.path import split
new_path = lambda p: '/'.join([n if n[:5]!='DeepQ' else 'DeepQ-Flipped' for n in p.split('/')])

import tqdm
for ip, lp in tqdm.tqdm(pathes, total=len(pathes)):
    with open(lp) as f:
        j = json.load(f)
    pathlib.Path(split(new_path(ip))[0]).mkdir(parents=True, exist_ok=True)
    pathlib.Path(split(new_path(lp))[0]).mkdir(parents=True, exist_ok=True)
    im = Image.open(ip).convert('RGB').transpose(Image.FLIP_LEFT_RIGHT)
    im.save(new_path(ip))
    with open(new_path(lp), 'w') as f:
        json.dump(json_flip(j, im), f, indent=2)
